In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing
import csv
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

pd.options.mode.chained_assignment = None
data = pd.read_csv('StatsAfterAllNBAInsertion.csv')

In [2]:
data.describe()

,Year,Age,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,MPpG,PER,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
count,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15095.000000,...,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000,15100.000000
mean,2004.197550,26.984834,0.027483,0.276358,0.430993,48.339603,23.073907,1130.345695,19.823603,12.399821,...,36.770662,0.642278,23.103245,0.398854,67.158808,1.190523,101.926821,1.864801,461.931854,7.824821
std,7.998757,4.149146,0.163493,1.305299,1.765006,26.478508,28.236547,915.995235,10.293082,6.282646,...,36.014443,0.466536,34.770722,0.497782,62.887391,0.794410,76.944305,0.851711,461.987449,5.834517
min,1990.000000,18.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-90.600000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1997.000000,24.000000,0.000000,0.000000,0.000000,25.000000,0.000000,292.000000,11.200000,9.600000,...,8.000000,0.300000,3.000000,0.100000,16.000000,0.600000,32.000000,1.200000,86.000000,3.300000
50%,2005.000000,26.000000,0.000000,0.000000,0.000000,52.000000,8.000000,940.000000,18.900000,12.600000,...,27.000000,0.600000,11.000000,0.200000,50.000000,1.000000,92.000000,1.900000,317.000000,6.300000
75%,2011.000000,30.000000,0.000000,0.000000,0.000000,74.000000,43.000000,1835.250000,28.300000,15.600000,...,55.000000,0.900000,28.000000,0.500000,102.000000,1.600000,159.000000,2.500000,713.250000,11.200000
max,2017.000000,44.000000,1.000000,15.000000,15.000000,85.000000,83.000000,3533.000000,44.500000,129.100000,...,246.000000,3.500000,376.000000,6.000000,464.000000,5.700000,371.000000,6.000000,2832.000000,35.400000


In [3]:
# Fill in blanks/NAs

data['PER'] = data['PER'].fillna(data['PER'].median())
data['TS%'] = data['TS%'].fillna(data['TS%'].median())
data['3PAr'] = data['3PAr'].fillna(data['3PAr'].median())
data['FTr'] = data['FTr'].fillna(data['FTr'].median())
data['ORB%'] = data['ORB%'].fillna(data['ORB%'].median())
data['DRB%'] = data['DRB%'].fillna(data['DRB%'].median())
data['TRB%'] = data['TRB%'].fillna(data['TRB%'].median())
data['AST%'] = data['AST%'].fillna(data['AST%'].median())
data['STL%'] = data['STL%'].fillna(data['STL%'].median())
data['BLK%'] = data['BLK%'].fillna(data['BLK%'].median())
data['TOV%'] = data['TOV%'].fillna(data['TOV%'].median())
data['USG%'] = data['USG%'].fillna(data['USG%'].median())
data['WS/48'] = data['WS/48'].fillna(data['WS/48'].median())
data['FG%'] = data['FG%'].fillna(data['FG%'].median())
data['3P%'] = data['3P%'].fillna(data['3P%'].median())
data['2P%'] = data['2P%'].fillna(data['2P%'].median())
data['eFG%'] = data['eFG%'].fillna(data['eFG%'].median())
data['FT%'] = data['FT%'].fillna(data['FT%'].median())


In [4]:
data.Pos.unique()

array(['PF', 'SF', 'C', 'SG', 'PG', 'SG-SF', 'PF-C', 'SF-PF', 'PF-SF',
       'SG-PG', 'PG-SG', 'C-PF', 'C-SF', 'SF-SG', 'PG-SF', 'SG-PF'],
      dtype=object)

In [5]:
#Give Position numerical labels

le = preprocessing.LabelEncoder()
data['Pos'] = le.fit_transform(data['Pos'].astype(str))
le_Pos_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_Pos_mapping)

{'C': 0, 'C-PF': 1, 'C-SF': 2, 'PF': 3, 'PF-C': 4, 'PF-SF': 5, 'PG': 6, 'PG-SF': 7, 'PG-SG': 8, 'SF': 9, 'SF-PF': 10, 'SF-SG': 11, 'SG': 12, 'SG-PF': 13, 'SG-PG': 14, 'SG-SF': 15}


In [6]:
# Delete duplicate Data that occured when Player transferred mid season

check = 0
index = 0
for year in data['Year']:
    if (check != year):
        check = year
        index = index + 1
        continue
    #remove row using index
    data = data.drop(index)
    index = index + 1

In [7]:
# Check if duplicate deletion was successful

def checkForDuplicates(dataframe):
    check = 0
    count = 0
    for year in dataframe['Year']:
        if (count == 2):
            return True
        if (check != year):
            check = year
            count = 0
            continue
        count = count + 1
        continue
    return False

In [8]:
print(checkForDuplicates(data))

False


In [9]:
# Split Dataset into sub-Datasets containing information about single players and careers per Dataframe

subDataSets = []
count = 0
string = 'A.C. Green'
for i,k in data.iterrows():
    if (string != k['Player']):
        subDataSets.append(data.loc[count:i-1])
        count = i
        string = k['Player']
    if (i == len(data)-1):
        subDataSets.append(data.loc[count:i])

In [10]:
# NOT Optional, only use players with at least 5 entries

careerPlayers = []
for subset in subDataSets:
    if (len(subset) >= 5):
        careerPlayers.append(subset)

In [11]:
# Loop through every single player's career and take averages of all features
# Use mode to determine the most played position
# Write result into a new csv file for further usage

with open('AverageStats.csv', 'w') as f1:
    for dataFrame in careerPlayers:
        dfTmp = dataFrame.groupby('Player').mean()
        dfTmp['Pos'] = dataFrame.mode()['Pos'][0]
        # Initially left out
        # dfTmp['Age'] = dataFrame.max()['Age']
        dfTmp.round(2).to_csv(f1, header=False)

In [12]:
# Open Average Stats over entire career include names of columns/headers

averageData = pd.read_csv('AverageStats.csv',names=['Player','Year','Age','Pos','All NBA','All NBA Acc','All NBA Total Career','G','GS','MP','MPpG','PER','TS%',
       '3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48','OBPM','DBPM','BPM',
       'VORP','FG','FGpG','FGA','FGApG','FG%','3P','3PpG','3PA','3PApG','3P%','2P','2PA','2P%','eFG%','FT','FTpG','FTA',
       'FTApG','FT%','ORB','ORBpG','DRB','DRBpG','TRB','TRBpG','AST','ASTpG','STL','STLpG','BLK','BLKpG','TOV','TOVpG',
       'PF','PFpG','PTS','PTSpG'])
averageData.head()

,Player,Year,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,A.C. Green,1995.50,31.50,3.0,0.0,0.0,0.0,79.42,51.08,2302.00,...,61.08,0.77,26.42,0.33,82.83,1.02,138.33,1.72,744.42,9.23
1,A.J. Price,2012.50,25.50,6.0,0.0,0.0,0.0,43.50,4.17,654.83,...,20.83,0.43,1.50,0.03,38.17,0.80,39.50,0.82,253.83,5.35
2,Aaron Brooks,2012.56,27.56,6.0,0.0,0.0,0.0,68.11,20.22,1472.22,...,39.22,0.57,9.67,0.14,109.89,1.56,132.11,1.89,687.11,9.61
3,Aaron Gray,2011.00,26.00,0.0,0.0,0.0,0.0,45.43,12.43,550.14,...,13.43,0.30,12.43,0.26,37.86,0.81,96.86,2.09,152.29,3.24
4,Aaron McKie,2001.00,28.00,12.0,0.0,0.0,0.0,61.00,22.08,1473.54,...,70.15,1.03,12.62,0.20,81.46,1.22,122.15,1.85,451.62,6.63


In [13]:
# Drop irrelevant features

averageData = averageData.drop('Player', axis=1)
averageData = averageData.drop('Year', axis=1)
# maybe drop age maybe not 
averageData = averageData.drop('Age', axis=1)
averageData = averageData.drop('All NBA', axis=1)
averageData = averageData.drop('All NBA Acc', axis=1)
averageData.head()

,Pos,All NBA Total Career,G,GS,MP,MPpG,PER,TS%,3PAr,FTr,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,3.0,0.0,79.42,51.08,2302.00,28.65,13.81,0.53,0.06,0.42,...,61.08,0.77,26.42,0.33,82.83,1.02,138.33,1.72,744.42,9.23
1,6.0,0.0,43.50,4.17,654.83,13.77,11.72,0.48,0.47,0.17,...,20.83,0.43,1.50,0.03,38.17,0.80,39.50,0.82,253.83,5.35
2,6.0,0.0,68.11,20.22,1472.22,20.84,12.79,0.52,0.41,0.19,...,39.22,0.57,9.67,0.14,109.89,1.56,132.11,1.89,687.11,9.61
3,0.0,0.0,45.43,12.43,550.14,11.99,11.56,0.53,0.00,0.38,...,13.43,0.30,12.43,0.26,37.86,0.81,96.86,2.09,152.29,3.24
4,12.0,0.0,61.00,22.08,1473.54,22.26,11.73,0.50,0.20,0.21,...,70.15,1.03,12.62,0.20,81.46,1.22,122.15,1.85,451.62,6.63


In [14]:
# Create correlation matrix

corr_matrix = averageData.corr().abs()

# Select upper triangle of correlation matrix

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95

to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

['MPpG',
 'TRB%',
 'WS',
 'FGpG',
 'FGA',
 'FGApG',
 '3PpG',
 '3PA',
 '3PApG',
 '2P',
 '2PA',
 'FTpG',
 'FTA',
 'FTApG',
 'ORBpG',
 'DRBpG',
 'TRB',
 'TRBpG',
 'ASTpG',
 'STLpG',
 'BLKpG',
 'TOVpG',
 'PTS',
 'PTSpG']

In [15]:
# Drop the highly corelated features from dataframe

for feature in to_drop:
    if (feature == 'All NBA Total Career'):
        continue
    averageData = averageData.drop(feature, axis=1)

In [16]:
averageData.describe()

,Pos,All NBA Total Career,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,...,FT,FT%,ORB,DRB,AST,STL,BLK,TOV,PF,PFpG
count,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,...,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000,1004.000000
mean,5.668327,0.433267,57.625488,28.545876,1380.437012,13.159373,0.515249,0.183785,0.315000,6.002759,...,106.851504,0.729791,68.786942,172.722430,125.005209,44.618685,28.347371,81.259920,124.141972,2.011753
std,4.237062,1.715721,12.295912,20.038302,603.454209,3.527359,0.041250,0.169380,0.143538,3.674822,...,81.985736,0.093903,50.675086,106.291126,115.057130,27.910667,29.473128,46.595156,46.264538,0.560968
min,0.000000,0.000000,12.710000,0.000000,74.860000,-1.050000,0.350000,0.000000,0.060000,0.900000,...,1.430000,0.320000,3.600000,10.200000,1.600000,1.800000,0.290000,4.290000,13.200000,0.500000
25%,3.000000,0.000000,50.692500,10.675000,909.887500,11.037500,0.490000,0.010000,0.220000,2.680000,...,48.192500,0.680000,30.950000,98.602500,45.100000,23.390000,9.400000,45.972500,91.595000,1.620000
50%,6.000000,0.000000,59.775000,25.180000,1346.500000,12.820000,0.520000,0.160000,0.290000,5.205000,...,84.490000,0.740000,55.070000,149.040000,89.000000,39.360000,19.055000,71.415000,123.515000,1.995000
75%,9.000000,0.000000,66.512500,43.800000,1836.525000,15.080000,0.540000,0.330000,0.390000,8.972500,...,143.507500,0.800000,95.895000,219.147500,166.195000,59.707500,35.632500,108.027500,156.337500,2.380000
max,12.000000,15.000000,81.200000,79.200000,2948.000000,27.700000,0.660000,0.780000,1.930000,19.140000,...,534.270000,0.920000,357.200000,669.000000,847.500000,196.800000,211.000000,291.110000,273.290000,3.870000


In [17]:
# Split dataframe into train and test sets aiming for ALL NBA TOTAL CAREER

X = averageData.drop("All NBA Total Career", axis=1)
y = averageData["All NBA Total Career"]

test_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42, shuffle=True)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(803, 38) (803,)
(201, 38) (201,)


In [18]:
# Train the linear regression algorithm using default parameters

r = LinearRegression()  
r.fit(X_train, y_train)
y_pred = r.predict(X_test)
df1 = pd.DataFrame({'Actual': y_test, 'Predicted All NBA': y_pred})
df1

,Actual,Predicted All NBA
926,0.0,-0.095252
630,0.0,1.243153
682,0.0,-0.484721
514,0.0,-0.154912
365,0.0,2.080030
655,0.0,-0.296726
656,0.0,0.956678
529,0.0,0.992840
321,0.0,-0.558625
70,0.0,-0.048937


In [19]:
r.coef_

array([ 1.82595031e-02,  5.68175092e-02,  5.69748048e-03, -4.40625769e-03,
        2.11157623e-01, -1.39436015e+01, -2.30805071e+00,  3.70117443e-01,
       -5.22210535e-02, -2.85715176e-02, -6.80170276e-02,  1.67380539e-01,
        3.02596026e-03,  4.07796082e-02, -1.82535250e-01,  2.22787413e-01,
        9.80646622e-01, -8.09215229e+00,  4.13642679e-01,  3.20949868e-02,
       -4.02168623e-01,  9.60976271e-01,  1.22904057e-02, -2.13781616e+01,
       -1.10533005e-02, -7.92246546e-01, -4.71197325e-01,  2.49549274e+01,
        4.88923874e-03, -1.12901304e+00, -6.99283497e-03,  3.10850676e-03,
        4.42860599e-03, -2.11487447e-02, -1.63420192e-03,  6.25844884e-03,
       -1.54557666e-02,  9.56677695e-01])

In [20]:
r2_score(y_test, y_pred) 

0.6431526859518055

In [21]:
mean_squared_error(y_test, y_pred)

0.7472056478607229

In [22]:
dataCurrent = pd.read_csv('2019Data.csv')
dataCurrent.head()

,Year,Player,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,2019,Luka Doncic,20,9,0,0,0,72,72,2318,...,77,1.1,25,0.3,247,3.4,137,1.9,1526,21.2
1,2020,Luka Doncic,20,9,0,0,0,17,17,577,...,25,1.5,3,0.2,79,4.6,44,2.6,512,30.1
2,2016,Karl-Anthony Towns,20,0,0,0,2,82,82,2627,...,58,0.7,138,1.7,183,2.2,245,3.0,1503,18.3
3,2017,Karl-Anthony Towns,21,0,0,0,2,82,82,3030,...,57,0.7,103,1.3,212,2.6,241,2.9,2061,25.1
4,2018,Karl-Anthony Towns,22,0,1,1,2,82,82,2918,...,64,0.8,115,1.4,159,1.9,285,3.5,1743,21.3


In [23]:
# Create new csv file with Towns', Irving's, Doncic's, and Curry's current career average stats  

with open('AverageStats2019.csv', 'w') as f2:
    dfTmp = dataCurrent.groupby('Player').mean()
    dfTmp.round(2).to_csv(f2, header=False)

In [24]:
# Open Average Stats 2019 over entire career include names of columns/headers

Average2019 = pd.read_csv('AverageStats2019.csv',names=['Player','Year','Age','Pos','All NBA','All NBA Acc','All NBA Total Career','G','GS','MP','MPpG','PER','TS%',
       '3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','OWS','DWS','WS','WS/48','OBPM','DBPM','BPM',
       'VORP','FG','FGpG','FGA','FGApG','FG%','3P','3PpG','3PA','3PApG','3P%','2P','2PA','2P%','eFG%','FT','FTpG','FTA',
       'FTApG','FT%','ORB','ORBpG','DRB','DRBpG','TRB','TRBpG','AST','ASTpG','STL','STLpG','BLK','BLKpG','TOV','TOVpG',
       'PF','PFpG','PTS','PTSpG'])
Average2019.head()

,Player,Year,Age,Pos,All NBA,All NBA Acc,All NBA Total Career,G,GS,MP,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,Karl-Anthony Towns,2018.0,21.80,0.0,0.60,1.00,2.0,67.80,67.80,2329.40,...,52.60,0.84,101.00,1.50,169.6,2.64,223.40,3.32,1521.40,23.08
1,Kyrie Irving,2016.0,23.22,6.0,0.33,0.89,2.0,57.67,57.67,1949.00,...,76.00,1.29,20.44,0.37,152.0,2.62,127.78,2.28,1289.78,17.92
2,Luka Doncic,2019.5,20.00,9.0,0.00,0.00,0.0,44.50,44.50,1447.50,...,51.00,1.30,14.00,0.25,163.0,4.00,90.50,2.25,1019.00,25.65
3,Stephen Curry,2015.0,26.00,6.0,0.64,2.45,6.0,63.45,62.91,2179.18,...,109.64,1.65,14.18,0.26,198.0,3.13,157.36,2.46,1490.55,22.80


In [25]:
# Drop irrelevant features

Average2019 = Average2019.drop('Player', axis=1)
Average2019 = Average2019.drop('Year', axis=1)
Average2019 = Average2019.drop('Age', axis=1)
Average2019 = Average2019.drop('All NBA', axis=1)
Average2019 = Average2019.drop('All NBA Acc', axis=1)
Average2019.head()

,Pos,All NBA Total Career,G,GS,MP,MPpG,PER,TS%,3PAr,FTr,...,STL,STLpG,BLK,BLKpG,TOV,TOVpG,PF,PFpG,PTS,PTSpG
0,0.0,2.0,67.80,67.80,2329.40,34.12,25.68,0.63,0.26,0.31,...,52.60,0.84,101.00,1.50,169.6,2.64,223.40,3.32,1521.40,23.08
1,6.0,2.0,57.67,57.67,1949.00,33.61,22.52,0.57,0.31,0.25,...,76.00,1.29,20.44,0.37,152.0,2.62,127.78,2.28,1289.78,17.92
2,9.0,0.0,44.50,44.50,1447.50,33.10,25.81,0.58,0.45,0.44,...,51.00,1.30,14.00,0.25,163.0,4.00,90.50,2.25,1019.00,25.65
3,6.0,6.0,63.45,62.91,2179.18,33.35,23.65,0.61,0.48,0.24,...,109.64,1.65,14.18,0.26,198.0,3.13,157.36,2.46,1490.55,22.80


In [26]:
# Drop previously highly correlated features from new dataframe

for feature in to_drop:
    if (feature == 'All NBA Total Career'):
        continue
    Average2019 = Average2019.drop(feature, axis=1)

In [27]:
X2019 = Average2019.drop('All NBA Total Career', axis=1)
y2019 = Average2019['All NBA Total Career']

In [36]:
# Use previous model to predict All NBA Total Career based on current career's stats

names = ['Towns', 'Irving', 'Doncic', 'Curry']
y2019_pred = r.predict(X2019)
df3 = pd.DataFrame({'Actual': names, 'Predicted All NBA Total': y2019_pred, 'All NBA Remaining': y2019_pred - y2019})
df3

,Actual,Predicted All NBA Total,All NBA Remaining
0,Towns,5.699087,3.699087
1,Irving,3.135502,1.135502
2,Doncic,1.906533,1.906533
3,Curry,5.136824,-0.863176


In [37]:
df3.loc[df3['All NBA Remaining'] < 0, 'All NBA Remaining'] = 0

In [38]:
df3.round()

,Actual,Predicted All NBA Total,All NBA Remaining
0,Towns,6.0,4.0
1,Irving,3.0,1.0
2,Doncic,2.0,2.0
3,Curry,5.0,0.0
